# Fast Empirical Survival Function Estimates

A notebook demonstrating how to make survival function estimates faster, based on the wickedly fast performance of Ben Gorman's `empirical_cdf()` function in his `mltools` package. 

Basically, I just used the same source code as his function, but reverse the coordinate order in the grid we are evaluating on (since evaluating the survival function from the bottom left is just like evaluating the cdf from the top right). It's so much faster than what I was doing before, as demonstrated here.

In [1]:
library(data.table)
library(mltools)
source('~/isolines_uq/scripts/R/auxiliary_scripts/utils.R')

Warning message:
“package ‘data.table’ was built under R version 4.2.3”
Warning message:
“package ‘mltools’ was built under R version 4.2.3”
Warning message:
“package ‘dplyr’ was built under R version 4.2.3”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“package ‘purrr’ was built under R version 4.2.3”

Attaching package: ‘purrr’


The following object is masked from ‘package:data.table’:

    transpose




In [21]:
lb <- -2
ub <- 7
gticks <- 400
grid <- expand.grid(X1 = seq(lb, ub, length.out=gticks), X2 = seq(lb, ub, length.out=gticks))

n <- 10000
dat <- data.frame(rmvt(n, sigma = matrix(c(1, 0.7, 0.7, 1), nrow = 2), df = 4))

In [22]:
start <- proc.time()
old_method <- apply(grid, 1, empSurv, dat=dat)
proc.time() - start

   user  system elapsed 
 21.316   0.212  21.539 

In [23]:
start <- proc.time()
new_method <- fastEmpSurv(grid, dat)
proc.time() - start

   user  system elapsed 
  0.196   0.000   0.110 

In [32]:
start <- proc.time()
grid_table <- data.table(grid)
dat_table <- data.table(dat)
ecdf <- empirical_cdf(dat_table, -grid_table)$CDF
proc.time() - start

   user  system elapsed 
  0.473   0.000   0.329 

In [33]:
all(old_method == new_method)

[1] TRUE

In [34]:
old_method

[1] 0.9108 0.9096 0.9087 0.9074 0.9062 0.9049 0.9036 0.9022 0.9009 0.8994
   [11] 0.8980 0.8965 0.8944 0.8926 0.8907 0.8899 0.8889 0.8867 0.8843 0.8822
   [21] 0.8796 0.8777 0.8749 0.8722 0.8689 0.8667 0.8646 0.8626 0.8592 0.8559
   [31] 0.8530 0.8501 0.8471 0.8437 0.8398 0.8358 0.8319 0.8282 0.8245 0.8205
   [41] 0.8162 0.8117 0.8075 0.8033 0.7981 0.7932 0.7885 0.7838 0.7800 0.7750
   [51] 0.7689 0.7652 0.7608 0.7561 0.7499 0.7438 0.7392 0.7323 0.7276 0.7219
   [61] 0.7155 0.7096 0.7035 0.6972 0.6899 0.6817 0.6750 0.6681 0.6617 0.6548
   [71] 0.6465 0.6402 0.6319 0.6238 0.6154 0.6077 0.5993 0.5917 0.5836 0.5748
   [81] 0.5670 0.5591 0.5513 0.5421 0.5352 0.5248 0.5176 0.5089 0.4995 0.4912
   [91] 0.4847 0.4761 0.4683 0.4607 0.4518 0.4438 0.4366 0.4281 0.4196 0.4122
  [101] 0.4039 0.3960 0.3879 0.3804 0.3719 0.3623 0.3557 0.3496 0.3420 0.3343
  [111] 0.3253 0.3163 0.3095 0.3025 0.2951 0.2891 0.2832 0.2775 0.2706 0.2646
  [121] 0.2582 0.2524 0.2460 0.2401 0.2343 0.2285 0.2237 0.2181 0.2135 0.2091
  [131] 0.2041 0.1977 0.1921 0.1885 0.1822 0.1779 0.1728 0.1684 0.1636 0.1593
  [141] 0.1553 0.1509 0.1474 0.1427 0.1386 0.1342 0.1297 0.1268 0.1234 0.1202
  [151] 0.1165 0.1134 0.1104 0.1081 0.1056 0.1034 0.1005 0.0976 0.0959 0.0935
  [161] 0.0909 0.0892 0.0866 0.0847 0.0820 0.0803 0.0776 0.0757 0.0736 0.0716
  [171] 0.0695 0.0669 0.0656 0.0642 0.0627 0.0610 0.0595 0.0575 0.0556 0.0541
  [181] 0.0531 0.0516 0.0503 0.0493 0.0474 0.0461 0.0448 0.0437 0.0427 0.0416
  [191] 0.0401 0.0395 0.0382 0.0376 0.0370 0.0359 0.0351 0.0345 0.0337 0.0329
  [201] 0.0323 0.0318 0.0309 0.0303 0.0293 0.0285 0.0283 0.0276 0.0268 0.0262
  [211] 0.0255 0.0250 0.0245 0.0245 0.0243 0.0236 0.0231 0.0228 0.0222 0.0217
  [221] 0.0213 0.0206 0.0202 0.0198 0.0189 0.0183 0.0180 0.0176 0.0173 0.0166
  [231] 0.0164 0.0160 0.0156 0.0153 0.0152 0.0147 0.0141 0.0136 0.0129 0.0124
  [241] 0.0122 0.0117 0.0117 0.0115 0.0111 0.0109 0.0108 0.0108 0.0105 0.0104
  [251] 0.0102 0.0099 0.0097 0.0097 0.0095 0.0093 0.0089 0.0089 0.0087 0.0087
  [261] 0.0086 0.0084 0.0081 0.0078 0.0076 0.0072 0.0070 0.0069 0.0068 0.0065
  [271] 0.0064 0.0064 0.0063 0.0063 0.0062 0.0060 0.0060 0.0059 0.0058 0.0057
  [281] 0.0055 0.0055 0.0054 0.0052 0.0052 0.0052 0.0050 0.0050 0.0049 0.0049
  [291] 0.0048 0.0047 0.0046 0.0045 0.0044 0.0043 0.0042 0.0040 0.0039 0.0039
  [301] 0.0038 0.0038 0.0037 0.0036 0.0036 0.0034 0.0034 0.0034 0.0034 0.0034
  [311] 0.0031 0.0029 0.0029 0.0028 0.0028 0.0028 0.0027 0.0025 0.0025 0.0024
  [321] 0.0024 0.0024 0.0023 0.0022 0.0021 0.0021 0.0020 0.0020 0.0020 0.0020
  [331] 0.0020 0.0020 0.0020 0.0020 0.0019 0.0019 0.0019 0.0019 0.0019 0.0017
  [341] 0.0017 0.0017 0.0016 0.0016 0.0016 0.0016 0.0016 0.0015 0.0015 0.0015
  [351] 0.0014 0.0014 0.0014 0.0014 0.0014 0.0014 0.0014 0.0013 0.0013 0.0013
  [361] 0.0013 0.0013 0.0012 0.0012 0.0012 0.0012 0.0012 0.0012 0.0012 0.0012
  [371] 0.0012 0.0012 0.0012 0.0012 0.0012 0.0012 0.0012 0.0012 0.0012 0.0012
  [381] 0.0011 0.0011 0.0011 0.0010 0.0010 0.0010 0.0010 0.0010 0.0010 0.0010
  [391] 0.0010 0.0009 0.0009 0.0009 0.0009 0.0008 0.0007 0.0007 0.0007 0.0007
  [401] 0.9095 0.9083 0.9074 0.9061 0.9049 0.9038 0.9025 0.9011 0.8998 0.8983
  [411] 0.8969 0.8954 0.8933 0.8915 0.8896 0.8889 0.8879 0.8858 0.8834 0.8813
  [421] 0.8787 0.8768 0.8740 0.8713 0.8680 0.8658 0.8638 0.8618 0.8584 0.8551
  [431] 0.8522 0.8493 0.8463 0.8429 0.8390 0.8350 0.8311 0.8274 0.8237 0.8197
  [441] 0.8155 0.8110 0.8068 0.8026 0.7975 0.7926 0.7880 0.7833 0.7795 0.7745
  [451] 0.7684 0.7647 0.7603 0.7556 0.7494 0.7433 0.7387 0.7318 0.7271 0.7214
  [461] 0.7150 0.7091 0.7031 0.6968 0.6895 0.6813 0.6746 0.6677 0.6613 0.6544
  [471] 0.6461 0.6398 0.6315 0.6234 0.6150 0.6073 0.5989 0.5913 0.5833 0.5745
  [481] 0.5667 0.5589 0.5511 0.5419 0.5350 0.5246 0.5174 0.5087 0.4993 0.4910
  [491] 0.4845 0.4759 0.4681 0.4605 0.4516 0.4436 0.4364 0.4279 0.4194 0.4120
  [501] 0.4037 0.3959 0.3878 0.3803 0.3718 0.3622 0.3556 0.3495 0.3419 0.3342
  [511] 0.3252 0.3162 0.30

In [35]:
grid

X1,X2
<dbl>,<dbl>
-2.000000,-2
-1.977444,-2
-1.954887,-2
-1.932331,-2
-1.909774,-2
-1.887218,-2
-1.864662,-2
-1.842105,-2
-1.819549,-2


In [47]:
empirical_cdf(dat_table, grid_table)

X1,X2,N.cum,CDF
<dbl>,<dbl>,<int>,<dbl>
-2.000000,-2,295,0.0295
-1.977444,-2,304,0.0304
-1.954887,-2,307,0.0307
-1.932331,-2,313,0.0313
-1.909774,-2,318,0.0318
-1.887218,-2,323,0.0323
-1.864662,-2,330,0.0330
-1.842105,-2,333,0.0333
-1.819549,-2,339,0.0339


In [48]:
mean(((dat)$X1 <= 7) & ((dat)$X2 <= 7))

[1] 0.9987

It recovers the exact same functions from the old method, and does so about 200 times faster..

## Playing around with the various steps

In [65]:
dat <- data.table(dat)
# flip the grid points around
grid <- data.table(grid[nrow(grid):1,])

In [66]:
grid

X1,X2
<dbl>,<dbl>
5.000000,5
4.979920,5
4.959839,5
4.939759,5
4.919679,5
4.899598,5
4.879518,5
4.859438,5
4.839357,5


In [47]:
dat_copy <- copy(dat[, names(grid), with=FALSE])

In [48]:
uboundDT <- unique(data.table(grid[['X1']], grid[['X1']]))
setnames(uboundDT, c('X1', paste0("Bound.X1")))
dat_copy <- uboundDT[dat_copy, on='X1', roll=Inf, nomatch=0]

In [49]:
uboundDT <- unique(data.table(grid[['X2']], grid[['X2']]))
setnames(uboundDT, c('X2', paste0("Bound.X2")))
dat_copy <- uboundDT[dat_copy, on='X2', roll=Inf, nomatch=0]

In [50]:
dat_copy

X2,Bound.X2,X1,Bound.X1
<dbl>,<dbl>,<dbl>,<dbl>
0.5680313,0.5622490,0.35909359,0.34136546
0.1947867,0.1807229,0.27291956,0.26104418
0.5806829,0.5622490,1.05803816,1.04417671
1.0849059,1.0843373,2.18433435,2.16867470
0.4719644,0.4618474,0.80956416,0.80321285
3.6341887,3.6144578,2.74495695,2.73092369
2.3033774,2.2891566,2.85367524,2.85140562
0.9261613,0.9236948,0.57335317,0.56224900
1.2382369,1.2248996,0.31351578,0.30120482


In [51]:
binned.uniques <- dat_copy[, .N, keyby=eval(paste0("Bound.", names(grid)))]

In [52]:
binned.uniques

Bound.X1,Bound.X2,N
<dbl>,<dbl>,<int>
0,0.00000000,2
0,0.02008032,1
0,0.04016064,3
0,0.06024096,2
0,0.08032129,1
0,0.14056225,1
0,0.16064257,3
0,0.18072289,4
0,0.20080321,3


In [53]:
setnames(binned.uniques, paste0("Bound.", names(grid)), names(grid))

In [54]:
binned.uniques

X1,X2,N
<dbl>,<dbl>,<int>
0,0.00000000,2
0,0.02008032,1
0,0.04016064,3
0,0.06024096,2
0,0.08032129,1
0,0.14056225,1
0,0.16064257,3
0,0.18072289,4
0,0.20080321,3


In [55]:
grid <- binned.uniques[grid, on=names(grid)]
grid[is.na(N), N := 0]

In [56]:
grid

X1,X2,N
<dbl>,<dbl>,<int>
5.000000,5,27
4.979920,5,0
4.959839,5,0
4.939759,5,2
4.919679,5,0
4.899598,5,0
4.879518,5,0
4.859438,5,0
4.839357,5,0


In [57]:
grid[, N.cum := cumsum(N), by='X2']

In [59]:
grid[, N.cum := cumsum(N.cum), by='X1']

In [60]:
grid

X1,X2,N,N.cum
<dbl>,<dbl>,<int>,<int>
5.000000,5,27,27
4.979920,5,0,27
4.959839,5,0,27
4.939759,5,2,29
4.919679,5,0,29
4.899598,5,0,29
4.879518,5,0,29
4.859438,5,0,29
4.839357,5,0,29


In [61]:
grid[, `:=`(N = NULL, Surv = N.cum/nrow(dat))]